In [28]:
from pymongo import MongoClient
import json
import pandas as pd
import numpy as np


client = MongoClient('mongodb://localhost:27017/')
db = client.companies


In [29]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [30]:
filtered_companies = db.filtered_companies.find()
companies_dislike = db.companies_dislike.find()

In [31]:
filtered_companies = pd.DataFrame(list(filtered_companies))
companies_dislike = pd.DataFrame(list(companies_dislike))

In [32]:
filtered_companies.head(3)

,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised,total_money_raised_USD
0,5d29ec7babbcf27c969c169c,software,None,Private Academic Networks,2008,"[{'source_url': '', 'raised_currency_code': 'U...",None,37.782263,-122.392142,GoingOn,40.0,"[{'city': 'San Francisco', 'zip_code': '94111'...","{'coordinates': [-122.392142, 37.782263], 'typ...","socialnetworking, enterprise, social-media, so...",1,$8.5M,8500000.0
1,5d29ec7babbcf27c969c169d,games_video,None,Digital Entertainment and Media Platform,2008,[{'source_url': 'http://www.deepjiveinterests....,None,37.781265,-122.393229,Crunchyroll,50.0,"[{'city': 'San Francisco', 'zip_code': '94105'...","{'coordinates': [-122.393229, 37.781265], 'typ...","crunchyroll, anime, manga, video, gaming, musi...",1,$4.8M,4800000.0
2,5d29ec7babbcf27c969c169e,enterprise,None,NoSQL database managed service,2008,"[{'source_url': '', 'raised_currency_code': 'U...",None,42.358920,-71.057810,Cloudant,70.0,"[{'city': 'Boston', 'zip_code': '02111', 'desc...","{'coordinates': [-71.05781, 42.35892], 'type':...","cloud-computing, big-data, databases, software...",1,$18.3M,18300000.0


In [33]:
companies_dislike.head()

,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised
0,5d29ec6dabbcf27c969c0195,web,None,None,1999,[],None,37.506885,-122.247573,Postini,NaN,"[{'city': 'San Carlos', 'zip_code': '94070', '...","{'coordinates': [-122.247573, 37.506885], 'typ...",,1,$0
1,5d29ec6dabbcf27c969c0196,web,None,Geneology social network site,2006,"[{'source_url': '', 'raised_currency_code': 'U...",None,34.090368,-118.393064,Geni,18.0,"[{'city': 'West Hollywood', 'zip_code': '90069...","{'coordinates': [-118.393064, 34.090368], 'typ...","geni, geneology, social, family, genealogy",1,$16.5M
2,5d29ec6dabbcf27c969c0197,games_video,None,None,2006,[{'source_url': 'http://www.lksf.org/eng/media...,None,40.746497,-74.009447,Joost,0.0,"[{'city': 'New York', 'zip_code': '10011-6903'...","{'coordinates': [-74.0094471, 40.7464969], 'ty...","iptv, babelgum, television, video, thevenicepr...",1,$45M
3,5d29ec6dabbcf27c969c0198,news,None,user driven social content website,2004,[{'source_url': 'http://www.marketingvox.com/a...,None,37.764726,-122.394523,Digg,60.0,"[{'city': 'San Francisco', 'zip_code': '94107'...","{'coordinates': [-122.394523, 37.764726], 'typ...","community, social, news, bookmark, digg, techn...",1,$45M
4,5d29ec6dabbcf27c969c0199,news,None,,1986,[],"{'pub_year': None, 'pub_day': None, 'valuation...",37.269175,-119.306607,CBS,NaN,"[{'city': '', 'zip_code': '', 'description': N...","{'coordinates': [-119.306607, 37.269175], 'typ...",None,1,$0


In [34]:
names_companies_dislike = list(companies_dislike['name'])

### Función para buscar las empresas que están en un radio determinado 

In [19]:
def findNear(geopoint, radio_max_meters=2000):
    return db.filtered_companies.find({"oficina_principal":{"$near":{
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })


radio_max_meters=2000
num_offices = pd.DataFrame()

In [20]:
punto = filtered_companies['oficina_principal'][8]
punto

{'coordinates': [5.291266, 52.132633], 'type': 'Point'}

In [21]:
lst = list(findNear(punto, radio_max_meters))
lst

[{'_id': ObjectId('5d29ec7babbcf27c969c16a4'),
  'category_code': 'hardware',
  'deadpooled_year': None,
  'description': 'Social Media Conglomerate',
  'founded_year': 2008,
  'funding_rounds': [{'funded_day': 1,
    'funded_month': 2,
    'funded_year': 2008,
    'id': 1586,
    'investments': [{'company': None,
      'financial_org': {'name': 'European Union',
       'permalink': 'european-union'},
      'person': None},
     {'company': {'name': 'BBC', 'permalink': 'bbc'},
      'financial_org': None,
      'person': None}],
    'raised_amount': 19000000,
    'raised_currency_code': 'EUR',
    'round_code': 'a',
    'source_description': '',
    'source_url': 'http://www.tribler.org/P2P-Next/19Million-for-P2P'}],
  'ipo': None,
  'lat': 52.132633,
  'lng': 5.291266,
  'name': 'P2P-Next',
  'number_of_employees': None,
  'offices': [{'address1': '',
    'address2': '',
    'city': '',
    'country_code': 'NLD',
    'description': None,
    'latitude': 52.132633,
    'longitude': 5.2

In [25]:
print(lst[0]['name'])

P2P-Next


In [40]:

for comp in filtered_companies['oficina_principal']:
    num = len(list(findNear(comp, radio_max_meters)))
    lst = list(findNear(comp, radio_max_meters))
    lst_of_company_names =[]
    for x in range(0,num):
        lst_of_company_names.append(lst[x]['name'])
        filtered_companies['offices_name_near'] = lst_of_company_names
lst_of_company_names

ValueError: Length of values does not match length of index

In [ ]:
names_companies_dislike

In [13]:
#Aqui hacer un ciclo for con la función dentro para que extraiga lo que quiero

lst_of_num_companies =[]
for comp in filtered_companies['oficina_principal']:
    lst_of_num_companies.append(len(list(findNear(comp, radio_max_meters))))


filtered_companies['offices_near']=lst_of_num_companies

In [19]:
definitive_companies = filtered_companies.sort_values('offices_near', ascending=False)

In [20]:
definitive_companies.head()


,_id,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,lat,lng,name,number_of_employees,offices,oficina_principal,tag_list,totalOffices,total_money_raised,offices_near
411,5d2701c60f9fc4a7eb3c37dc,ecommerce,None,Spa and Well-being social network,2008,"[{'round_code': 'angel', 'investments': [], 's...",None,37.786910,-122.404403,InnerRewards,9.0,"[{'description': 'West Coast', 'city': 'San Fr...","{'coordinates': [-122.404403, 37.78691], 'type...","wellness, women, social-networking, spas, ecom...",2,$1M,77
493,5d2701c60f9fc4a7eb3c3839,ecommerce,None,Spa and Well-being social network,2008,"[{'round_code': 'angel', 'investments': [], 's...",None,37.786910,-122.404403,InnerRewards,9.0,"[{'description': 'West Coast', 'city': 'San Fr...","{'coordinates': [-122.404403, 37.78691], 'type...","wellness, women, social-networking, spas, ecom...",2,$1M,77
1277,5d2701c60f9fc4a7eb3c3b9c,biotech,None,Cancer management,2008,"[{'round_code': 'a', 'investments': [{'company...",None,37.787131,-122.404107,ACT Biotech,NaN,"[{'description': '', 'city': 'San Francisco', ...","{'coordinates': [-122.4041075, 37.7871306], 't...","biotech, development, cancer, drugs, medical, ...",1,$27M,77
871,5d2701c60f9fc4a7eb3c39d4,web,None,Social Recruiting and Job Search Site,2008,"[{'round_code': 'angel', 'investments': [], 's...",None,37.788796,-122.409710,KODA,15.0,"[{'description': 'San Francisco', 'city': 'San...","{'coordinates': [-122.40971, 37.788796], 'type...",,2,$4.5M,76
38,5d2701c60f9fc4a7eb3c3657,web,None,None,2008,[],None,37.780951,-122.402502,Heavy Electrons,NaN,"[{'description': '', 'city': 'San Francisco', ...","{'coordinates': [-122.4025016, 37.7809512], 't...",,1,$0,71


['$16.8M', '$31.4M', '$46.8M', '$5.46M', 'C$350k', '$190M', '$2.35M', '$100k', '$16k', '$10.1M', '$25k', '$110k', '$5.9M', '$68.8M', '€1.1M', '$500k', '€424k', '$16.3M', '€19M', '$8.09M', '$23M', '$15M', '$28.4M', '$8.5M', '$20k', '$141M', '$25M', '$7.84M', '$1.1M', '$2.8M', '$930k', '$55k', '$89.5M', '€141k', '$8.25M', '$24M', '$357M', '$1.3M', '$3.8M', '$78.1M', '$3.1M', '$3.23M', '$5.1M', '$75k', '$6.66M', '$22.1M', '$4.2M', '$17.2M', '$270k', '$25.5M', '€500k', '$7M', '$44.5M', '$6.5M', '$650k', '$55M', '£1M', '$900k', '$13M', '$1.83M', '£3.4M', '$150k', '$57.3M', '$16M', '$10.2M', '$600k', '$4.8M', '$7.5M', '$14.5M', '$42.5M', '$550k', '$15k', '$48M', '$7.1M', '$11M', '$3.01M', '$16.6M', '€1.13M', '$8.1M', '$35M', '$1.6M', '$6.64M', '$5M', '$45M', '$50k', '$505k', '$59.5M', '$33.2M', '$7.55M', '$1.77M', '$1.25M', '$7.3M', '$1.7M', '$2.98M', '$1.08M', '$5.5M', '$6M', '$2.75M', '$80k', '$4.5M', '$2.5M', 'C$325k', '$400k', '€3.5M', '€70k', '$26.4M', '$275k', '£7.66M', '$11.5M', '€25M

In [ ]:
definitive_companies.to_json('./definitive_companies.json', orient='records', default_handler=str)